## Get data downloaded

In [ ]:
!wget -P /mnt/data/machine-learning http://storage.yandexcloud.net/covid19.1110/prod/COVID19_1110.zip
!unzip /mnt/data/machine-learning/COVID19_1110.zip -d /mnt/LSDF/users/zharov

## Prepare unsupervised dataset

In [1]:
!mkdir /mnt/LSDF/users/zharov/COVID19_1110/studies/prepared

mkdir: cannot create directory ‘/mnt/LSDF/users/zharov/COVID19_1110/studies/prepared’: File exists


In [1]:
import tifffile
from medpy.io import load as medload
import numpy as np
from glob import glob
from skimage.exposure import rescale_intensity
from tqdm.auto import tqdm
import os
import re

inner_medload = lambda a: np.swapaxes(medload(a)[0], 0, -1)

In [3]:
all_addrs = glob('/mnt/LSDF/users/zharov/COVID19_1110/studies/CT-*/*.nii.gz')

In [4]:
extract_experiment_id = re.compile(r'study_(\d+).nii.gz')

In [5]:
for addr in tqdm(all_addrs):
    img = inner_medload(addr)
    filename = os.path.split(addr)[1]
    experiment_id = extract_experiment_id.findall(filename)[0]
    img = rescale_intensity(img, in_range=(-500, 500), out_range=(0, 255)).astype(np.uint8)
    tifffile.imsave(f'/mnt/LSDF/users/zharov/COVID19_1110/studies/prepared/{experiment_id}.tiff', img)

## Prepare supervised dataset

In [3]:
import shutil

In [4]:
extract_mask_id = re.compile(r'study_(\d+)_mask.nii.gz')

In [5]:
!mkdir /mnt/LSDF/users/zharov/COVID19_1110/uniformly_named_masks

In [7]:
for addr in tqdm(glob('/mnt/LSDF/users/zharov/COVID19_1110/masks/*.nii.gz')):
    id = extract_mask_id.findall(os.path.split(addr)[1])[0]
    shutil.copy(addr, f'/mnt/LSDF/users/zharov/COVID19_1110/uniformly_named_masks/{id}.nii.gz')

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
len(glob('/mnt/LSDF/users/zharov/COVID19_1110/uniformly_named_masks/*.nii.gz'))

50

In [9]:
msk = inner_medload(addr)

In [13]:
msk.shape, msk.dtype, np.unique(msk, return_counts=True)

((42, 512, 512),
 dtype('int16'),
 (array([0, 1], dtype=int16), array([10993839,    16209])))